In [1]:
import pandas as pd
import yaml
from functions import trim_and_lower

In [2]:
try:
    with open("../config.yaml", "r") as file:
        config = yaml.safe_load(file)
except:
    print("The configuration file was not found!")

***Notes regarding the web data below***
1. Some users look to have entered the process after the start
2. Some users entered the process after start and then went back to start
3. There are situations where the same visit_id is shared by different client_ids

***Groups***
- Those sessions that followed all steps consecutively
- Those sessions that did not complete the whole process and/or encountered errors
- Those sessions including duplicate actions

In [3]:
# Load the dataframes

groups_path = config['data']['raw']['raw_2']
groups = pd.read_csv(groups_path)
groups = trim_and_lower(groups)

path = config['data']['raw']['raw_3']
web_data_1 = pd.read_csv(path)

path = config['data']['raw']['raw_4']
web_data_2 = pd.read_csv(path)

In [4]:
# Combine and format the dataframes

df = pd.concat([web_data_1, web_data_2], axis=0)
df = trim_and_lower(df)
df.date_time = pd.to_datetime(df.date_time)
df = df.merge(groups, on='client_id', how='left')

# Drop rows for non A/B test clients.
df = df[df.variation.notnull()]
df.head()

,client_id,visitor_id,visit_id,process_step,date_time,variation
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07,test
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51,test
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22,test
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13,test
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04,test


In [5]:
repeated_steps = df.groupby(['client_id', 'visit_id', 'process_step', 'variation']).size().reset_index(name='count')
repeated_steps['repeated'] = repeated_steps['count'].apply(lambda x: 'no' if x == 1 else 'yes')
repeated_steps

,client_id,visit_id,process_step,variation,count,repeated
0,555,637149525_38041617439_716659,confirm,test,1,no
1,555,637149525_38041617439_716659,start,test,1,no
2,555,637149525_38041617439_716659,step_1,test,1,no
3,555,637149525_38041617439_716659,step_2,test,1,no
4,555,637149525_38041617439_716659,step_3,test,1,no
...,...,...,...,...,...,...
239326,9999729,870243567_56915814033_814203,step_3,test,1,no
239327,9999729,99583652_41711450505_426179,start,test,1,no
239328,9999729,99583652_41711450505_426179,step_1,test,1,no
239329,9999832,472154369_16714624241_585315,start,test,1,no


In [6]:
repeated_steps.repeated.value_counts()

repeated
no     188681
yes     50650
Name: count, dtype: int64

In [7]:
# Create a funnel based on the number of occurences of each process_step.

funnel = df.pivot_table(index='process_step', columns='variation', values='client_id', aggfunc='count').sort_values(by='test', ascending=False)
funnel

variation,control,test
process_step,,
start,47608,56474
step_1,29668,38768
step_2,25886,30989
step_3,22795,25894
confirm,17505,25722


In [8]:
# Create a dataframe showing the outcome for each client_id/visit_id.

complete_set = set(df[df['process_step'] == 'confirm'][['client_id', 'visit_id']].apply(tuple, axis=1))
session_outcome = df.copy()[['client_id', 'visit_id', 'variation']].drop_duplicates().reset_index(drop=True)
session_outcome['process_completed'] = session_outcome.apply(lambda row: 'yes' if (row['client_id'], row['visit_id']) in complete_set else 'no', axis=1)
session_outcome.head()

,client_id,visit_id,variation,process_completed
0,9988021,781255054_21935453173_531117,test,no
1,8320017,960651974_70596002104_312201,test,yes
2,4033851,949661017_22392791362_127391,control,yes
3,1982004,132494692_73738638511_70823,test,yes
4,9294070,116640698_27328181095_188045,control,no


In [9]:
# Group and count occurrences of each process step for each client_id/visit_id combination.
count_df = df.groupby(['client_id', 'visit_id', 'process_step', 'variation']).size().reset_index(name='count')


# Create a dataframe using pivot table showing the count of each step for each client_id/visit_id combination.
count_df_pivot = count_df.pivot_table(index=['client_id', 'visit_id'], columns='process_step', values='count', aggfunc='first').reset_index()
count_df_pivot

process_step,client_id,visit_id,confirm,start,step_1,step_2,step_3
0,555,637149525_38041617439_716659,1.0,1.0,1.0,1.0,1.0
1,647,40369564_40101682850_311847,1.0,1.0,1.0,1.0,1.0
2,934,7076463_57954418406_971348,NaN,4.0,NaN,NaN,NaN
3,1028,557292053_87239438319_391157,NaN,1.0,5.0,2.0,1.0
4,1104,543158812_46395476577_767725,NaN,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...
69442,9999626,182723364_27031318473_880901,NaN,1.0,1.0,NaN,NaN
69443,9999729,493310979_9209676464_421146,NaN,2.0,1.0,1.0,NaN
69444,9999729,870243567_56915814033_814203,1.0,1.0,1.0,1.0,1.0
69445,9999729,99583652_41711450505_426179,NaN,1.0,1.0,NaN,NaN


In [10]:
# Drop all rows where all steps in the process were taken. This leaves us with only client sessions including repeated and missed steps.
sessions_with_errors = count_df_pivot[~((count_df_pivot['start'] == 1) & (count_df_pivot['step_1'] == 1) & (count_df_pivot['step_2'] == 1) & (count_df_pivot['step_3'] == 1) & (count_df_pivot['confirm'] == 1))]
errors_set = set(sessions_with_errors[['client_id', 'visit_id']].apply(tuple, axis=1))

# Apply a function to create the 'error_in_session' column.
#session_outcome['error_in_session'] = df.apply(lambda row: 'yes' if (row['client_id'], row['visit_id']) in errors_set else 'no', axis=1)

In [11]:
# Apply a function to create the 'error_in_session' column with debugging output
def check_error(row):
    tuple_check = (row['client_id'], row['visit_id'])
    in_errors_set = tuple_check in errors_set
    return 'yes' if in_errors_set else 'no'

session_outcome['error_in_session'] = session_outcome.apply(check_error, axis=1)


In [12]:
# Pivot the DataFrame
df_copy = df.copy().sort_values(by='date_time')
pivot_table = df_copy.pivot_table(
    index=['client_id', 'visitor_id', 'visit_id'],
    columns='process_step',
    values='date_time',
    aggfunc='first'
).reset_index()

pivot_table.columns.name = None

# Sort columns
pivot_table = pivot_table[['client_id', 'visitor_id', 'visit_id', 'start', 'step_1', 'step_2', 'step_3', 'confirm']]
pivot_table

,client_id,visitor_id,visit_id,start,step_1,step_2,step_3,confirm
0,555,402506806_56087378777,637149525_38041617439_716659,2017-04-15 12:57:56,2017-04-15 12:58:03,2017-04-15 12:58:35,2017-04-15 13:00:14,2017-04-15 13:00:34
1,647,66758770_53988066587,40369564_40101682850_311847,2017-04-12 15:41:28,2017-04-12 15:41:35,2017-04-12 15:41:53,2017-04-12 15:45:02,2017-04-12 15:47:45
2,934,810392784_45004760546,7076463_57954418406_971348,2017-04-18 02:36:30,NaT,NaT,NaT,NaT
3,1028,42237450_62128060588,557292053_87239438319_391157,2017-04-08 18:51:28,2017-04-08 18:52:17,2017-04-08 18:53:29,2017-04-08 18:58:04,NaT
4,1104,194240915_18158000533,543158812_46395476577_767725,2017-06-12 07:49:18,NaT,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...
69442,9999626,52633065_71189986073,182723364_27031318473_880901,2017-05-14 09:07:51,2017-05-14 09:07:59,NaT,NaT,NaT
69443,9999729,604429154_69247391147,99583652_41711450505_426179,2017-04-05 13:40:49,2017-04-05 13:41:04,NaT,NaT,NaT
69444,9999729,834634258_21862004160,870243567_56915814033_814203,2017-05-08 16:08:25,2017-05-08 16:08:30,2017-05-08 16:08:40,2017-05-08 16:09:19,2017-05-08 16:09:40
69445,9999729,843385170_36953471821,493310979_9209676464_421146,2017-04-20 14:21:27,2017-04-20 14:22:49,2017-04-20 14:27:36,NaT,NaT


In [13]:
pivot_table['s1_ttc'] = (pivot_table.step_1 - pivot_table.start).dt.total_seconds()
pivot_table['s2_ttc'] = (pivot_table.step_2 - pivot_table.step_1).dt.total_seconds()
pivot_table['s3_ttc'] = (pivot_table.step_3 - pivot_table.step_2).dt.total_seconds()
pivot_table['confirm_ttc'] = (pivot_table.confirm - pivot_table.step_3).dt.total_seconds()
pivot_table['total_ttc'] = pivot_table.s1_ttc + pivot_table.s2_ttc + pivot_table.s3_ttc + pivot_table.confirm_ttc
pivot_table = pivot_table.drop(columns=['start', 'step_1', 'step_2', 'step_3', 'confirm'])
pivot_table

,client_id,visitor_id,visit_id,s1_ttc,s2_ttc,s3_ttc,confirm_ttc,total_ttc
0,555,402506806_56087378777,637149525_38041617439_716659,7.0,32.0,99.0,20.0,158.0
1,647,66758770_53988066587,40369564_40101682850_311847,7.0,18.0,189.0,163.0,377.0
2,934,810392784_45004760546,7076463_57954418406_971348,NaN,NaN,NaN,NaN,NaN
3,1028,42237450_62128060588,557292053_87239438319_391157,49.0,72.0,275.0,NaN,NaN
4,1104,194240915_18158000533,543158812_46395476577_767725,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
69442,9999626,52633065_71189986073,182723364_27031318473_880901,8.0,NaN,NaN,NaN,NaN
69443,9999729,604429154_69247391147,99583652_41711450505_426179,15.0,NaN,NaN,NaN,NaN
69444,9999729,834634258_21862004160,870243567_56915814033_814203,5.0,10.0,39.0,21.0,75.0
69445,9999729,843385170_36953471821,493310979_9209676464_421146,82.0,287.0,NaN,NaN,NaN


In [14]:
# Export df, session_outcome and pivot_table

output_file_path = config['data']['clean']['clean_2']
df.to_csv(output_file_path, index=False)

output_file_path = config['data']['clean']['clean_3']
session_outcome.to_csv(output_file_path, index=False)

output_file_path = config['data']['clean']['clean_4']
pivot_table.to_csv(output_file_path, index=False)

output_file_path = config['data']['clean']['clean_6']
repeated_steps.to_csv(output_file_path, index=False)